In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
import warnings
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from torch.utils.data import TensorDataset, DataLoader
warnings.filterwarnings('ignore')

In [13]:
class Config:
    DATA_DIR = Path("./data")
    OUTPUT_DIR = Path("./outputs")
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 200
    PATIENCE = 30
    LEARNING_RATE = 1e-3
    
    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)

In [ ]:
def feature_engineering(input_df):
    """
    Perform feature engineering on the input DataFrame.
    
    Args:
        input_df (pd.DataFrame): Input DataFrame containing raw tracking data
    """
    pass


In [ ]:
def data_preparation(input_df, output_df):
    """
    Prepare the input DataFrame for model training.
    
    Args:
        input_df (pd.DataFrame): Input DataFrame containing raw tracking data
        output_df (pd.DataFrame): Output DataFrame containing target data
    """
    pass


In [ ]:
def model_training(train_input, train_output, test_input, test_output):

In [6]:
def get_opponent_proximity_simple(input_df):
    """Only basic opponent proximity - no complex features"""
    features = []
    
    for (gid, pid), group in tqdm(input_df.groupby(['game_id', 'play_id']), 
                                   desc="🏈 Opponent proximity", leave=False):
        last = group.sort_values('frame_id').groupby('nfl_id').last()
        
        if len(last) < 2:
            continue
            
        positions = last[['x', 'y']].values
        sides = last['player_side'].values
        speeds = last['s'].values
        directions = last['dir'].values
        
        for i, (nid, side) in enumerate(zip(last.index, sides)):
            opp_mask = sides != side
            
            feat = {
                'game_id': gid, 'play_id': pid, 'nfl_id': nid,
                'nearest_opp_dist': 50.0,
                'num_nearby_opp_3': 0,
                'num_nearby_opp_5': 0,
                'closing_speed_opp': 0.0,
            }

            if not opp_mask.any():
                features.append(feat)
                continue
            
            opp_positions = positions[opp_mask]
            distances = np.sqrt(((positions[i] - opp_positions)**2).sum(axis=1))
            
            if len(distances) == 0:
                features.append(feat)
                continue
                
            nearest_idx = distances.argmin()
            feat['nearest_opp_dist'] = distances[nearest_idx]
            feat['num_nearby_opp_3'] = (distances < 3.0).sum()
            feat['num_nearby_opp_5'] = (distances < 5.0).sum()
            
            # Simple closing speed
            my_vx = speeds[i] * np.sin(np.deg2rad(directions[i]))
            my_vy = speeds[i] * np.cos(np.deg2rad(directions[i]))
            opp_speeds = speeds[opp_mask]
            opp_dirs = directions[opp_mask]
            opp_vx = opp_speeds[nearest_idx] * np.sin(np.deg2rad(opp_dirs[nearest_idx]))
            opp_vy = opp_speeds[nearest_idx] * np.cos(np.deg2rad(opp_dirs[nearest_idx]))
            
            rel_vx = my_vx - opp_vx
            rel_vy = my_vy - opp_vy
            to_me = positions[i] - opp_positions[nearest_idx]
            to_me_norm = to_me / (np.linalg.norm(to_me) + 0.1)
            feat['closing_speed_opp'] = -(rel_vx * to_me_norm[0] + rel_vy * to_me_norm[1])

            
            features.append(feat)
    
    return pd.DataFrame(features)
            
        

In [5]:
def add_safe_catboost_features(df):
    """🎯 ONLY SAFE, GENERALIZABLE ADDITIONS"""
    print("Adding safe CatBoost features (physics-based only)...")
    
    # Player BMI
    height_parts = df['player_height'].str.split('-', expand=True)
    df['height_inches'] = height_parts[0].astype(float) * 12 + height_parts[1].astype(float)
    df['bmi'] = (df['player_weight'] / (df['height_inches']**2)) * 703
    
    # Orientation alignment
    df['orientation_diff'] = np.abs(df['o'] - df['dir'])
    df['orientation_diff'] = np.minimum(df['orientation_diff'], 360 - df['orientation_diff'])
    
    # Non-linear physics
    df['speed_squared'] = df['s'] ** 2
    df['dist_squared'] = df['distance_to_ball'] ** 2
    
    # Angle to target
    df['angle_diff'] = np.abs(df['o'] - np.degrees(df['angle_to_ball']))
    df['angle_diff'] = np.minimum(df['angle_diff'], 360 - df['angle_diff'])
    
    # Better closing speed calculation
    df['velocity_toward_ball'] = (
        df['velocity_x'] * np.cos(df['angle_to_ball']) + 
        df['velocity_y'] * np.sin(df['angle_to_ball'])
    )
    
    print("Added 6 safe physics features")
    return df

In [17]:
config = Config()

print("\n[1/4] Loading data...")
train_input_files = [config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, 19)]
train_output_files = [config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]
train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])
test_input = pd.read_csv(config.DATA_DIR / "test_input.csv")
test_template = pd.read_csv(config.DATA_DIR / "test.csv")



[1/4] Loading data...
